<a href="https://colab.research.google.com/github/aptyp78/PIKAi/blob/colab-latest/notebooks/Grounded_DINO_SAM2_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GroundedDINO + SAM — Detection (Colab Pro+) — commit afaa196 — 2025-09-16T18:30:50Z
Детектор регионов: монтируем GCS (через сервис‑аккаунт), ставим Torch+детекторы, рендерим страницы, запускаем детекцию и грузим регионы в `gs://pik-artifacts-dev/grounded_regions/`.


In [1]:
#@title No-op gate helper
def require_start():
  return None


In [2]:
#@title Cell Execution Logger
import os, sys, json, time, uuid, warnings
from pathlib import Path
try:
  LOG_DIR  # noqa: F821
except NameError:
  RUN_ID = time.strftime('%Y%m%d-%H%M%S')
  LOCAL_LOG_ROOT = '/content/colab_runs'
  LOG_DIR = Path(LOCAL_LOG_ROOT)/RUN_ID
  LOG_DIR.mkdir(parents=True, exist_ok=True)

from IPython import get_ipython
ip = get_ipython()

class _Tee:
  def __init__(self, stream, buf_list):
    self._s = stream; self._b = buf_list
  def write(self, s):
    try: self._s.write(s)
    finally: self._b.append(s)
  def flush(self):
    try: self._s.flush()
    except Exception: pass

_celllog = {'i': None, 'start': None, 'buf_out':[], 'buf_err':[], 'warns':[], 'id': None}
_orig_out, _orig_err = sys.stdout, sys.stderr
_orig_showwarning = warnings.showwarning
LOG_JSONL = str(LOG_DIR/'cells.jsonl')

def _pre(cell_id):
  _celllog['i'] = ip.execution_count + 1
  _celllog['id'] = str(uuid.uuid4())
  _celllog['start'] = time.time()
  _celllog['buf_out'] = []
  _celllog['buf_err'] = []
  _celllog['warns'] = []
  sys.stdout = _Tee(_orig_out, _celllog['buf_out'])
  sys.stderr = _Tee(_orig_err, _celllog['buf_err'])
  def _sw(message, category, filename, lineno, file=None, line=None):
    _celllog['warns'].append({'message': str(message), 'category': getattr(category,'__name__', str(category)), 'filename': filename, 'lineno': lineno})
    return _orig_showwarning(message, category, filename, lineno, file, line)
  warnings.showwarning = _sw

def _post(result):
  # restore
  sys.stdout = _orig_out
  sys.stderr = _orig_err
  warnings.showwarning = _orig_showwarning
  end = time.time()
  i = _celllog.get('i')
  # Try to get cell source from history
  src = None
  try:
    ih = ip.user_ns.get('_ih', [])
    if i is not None and i < len(ih):
      src = ih[i]
  except Exception:
    src = None
  rec = {
    'cell_id': _celllog.get('id'),
    'execution_count': i,
    'start_ts': _celllog.get('start'),
    'end_ts': end,
    'duration_s': (end - _celllog['start']) if _celllog.get('start') else None,
    'success': bool(getattr(result, 'success', True)),
    'out': ''.join(_celllog.get('buf_out') or []),
    'err': ''.join(_celllog.get('buf_err') or []),
    'warnings': _celllog.get('warns') or [],
    'source': src,
  }
  try:
    with open(LOG_JSONL, 'a', encoding='utf-8') as f:
      f.write(json.dumps(rec, ensure_ascii=False) + '\n')
  except Exception as e:
    print('[cell-logger] write failed:', e)

ip.events.register('pre_run_cell', _pre)
ip.events.register('post_run_cell', _post)
print('[cell-logger] enabled ->', LOG_JSONL)


[cell-logger] enabled -> /content/colab_runs/20250916-161108/cells.jsonl


In [3]:
#@title Runtime & GPU
NOTEBOOK_VERSION = 'afaa196'
print('Notebook version:', NOTEBOOK_VERSION)
# Runtime & GPU
!nvidia-smi || true
import sys; print(sys.version)
NOTEBOOK_UPDATED = '2025-09-16T18:30:50Z'


Notebook version: 2652b08
Tue Sep 16 16:11:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------

In [4]:
#@title Auth + gcsfuse setup
require_start()

# Install packages and prepare gcsfuse repo; auto-mount with SA from /content/Secrets if present
# Try Colab user auth (optional)
try:
  from google.colab import auth
  auth.authenticate_user()
  print('[auth] Colab user credentials OK')
except Exception as e:
  print('[auth] Skipping Colab user auth:', e)
!pip -q install google-cloud-storage gcsfs==2025.3.0 fsspec==2025.3.0
!sudo install -m 0755 -d /usr/share/keyrings
!curl -fsSL https://packages.cloud.google.com/apt/doc/apt-key.gpg -o /tmp/cloud.google.gpg
!sudo gpg --dearmor --yes --batch -o /usr/share/keyrings/cloud.google.gpg /tmp/cloud.google.gpg || sudo cp /tmp/cloud.google.gpg /usr/share/keyrings/cloud.google.gpg
!echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt gcsfuse-jammy main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list >/dev/null
!sudo apt-get -q update
!sudo apt-get -q install -y gcsfuse poppler-utils
!mkdir -p /content/src_gcs /content/artifacts /content/pages /content/Secrets
import glob, os, subprocess, shlex

# Read SA key from Colab Secrets (GCS_SA_JSON / GCS_SA_JSON2 / secretName) and optional GOOGLE_API_KEY
try:
  from google.colab import userdata as _ud
  _sa = None; _sa_name = ''
  for _k in ('GCS_SA_JSON','GCS_SA_JSON2','secretName'):
    try:
      _v = _ud.get(_k)
    except Exception:
      _v = None
    if _v:
      _sa = _v; _sa_name = _k; break
  try:
    _gapi = _ud.get('GOOGLE_API_KEY')
  except Exception:
    _gapi = None
except Exception:
  _sa = None; _gapi = None; _sa_name = ''
if _gapi:
  os.environ['GOOGLE_API_KEY'] = _gapi
  print('[auth] GOOGLE_API_KEY loaded from Colab Secrets')
if _sa:
  os.makedirs('/content/Secrets', exist_ok=True)
  _key_path = '/content/Secrets/sa.json'
  open(_key_path,'w',encoding='utf-8').write(_sa)
  os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = _key_path
  print('[auth] SA key written to', _key_path, '(source:', _sa_name, ')')

# Skip auto-mount here; use the next cell 'Mount GCS buckets'
print('[auth] SA prepared. Proceed to mount in the next cell.')


[auth] Colab user credentials OK
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease [1,227 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:12 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:13 http://archive.ubuntu.com/ubuntu 

In [5]:
#@title Compatibility Fixes (pip pins)
require_start()
# Do not upgrade ipython (Colab expects ipython==7.34.0)
# Remove xformers if present (version coupling to torch; optional)
!pip -q uninstall -y xformers || true

# Pin compatible versions for known conflicts
!pip -q install -U   "typing_extensions>=4.14.0,<5"   "filelock>=3.15"   "numpy<2.1,>=1.24"   gcsfs==2025.3.0 fsspec==2025.3.0

from importlib import metadata as md
def _ver(name, mod=None):
    try:
        return md.version(name)
    except Exception:
        try:
            m = __import__(mod or name)
            return getattr(m, "__version__", "unknown")
        except Exception:
            return "not installed"
print(
  "[compat]",
  "jedi=", _ver("jedi"),
  "typing_extensions=", _ver("typing_extensions","typing_extensions"),
  "filelock=", _ver("filelock"),
  "numpy=", _ver("numpy"),
  "gcsfs=", _ver("gcsfs"),
  "fsspec=", _ver("fsspec"),
)


[compat] jedi= not installed typing_extensions= 4.15.0 filelock= 3.19.1 numpy= 2.0.2 gcsfs= 2025.3.0 fsspec= 2025.3.0


In [6]:
#@title Mount GCS buckets
require_start()

# Robust mount with verbose logs and fallback info
import os, glob, subprocess, pathlib, textwrap
pathlib.Path('/content/src_gcs').mkdir(parents=True, exist_ok=True)
pathlib.Path('/content/artifacts').mkdir(parents=True, exist_ok=True)
pathlib.Path('/content/gcsfuse_tmp').mkdir(parents=True, exist_ok=True)
key = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', '')
if (not key) and 'KEY' in globals(): key = KEY
if key and not os.path.isabs(key): key = os.path.join('/content', key)
if not (key and os.path.exists(key)):
  matches = glob.glob('/content/Secrets/*.json')
  if matches:
    key = matches[0]; os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key
  else:
    raise SystemExit('Service account key not found — upload to /content/Secrets/*.json')
print('Using SA key:', key)
def mount(bucket, mnt):
  log=f'/content/gcsfuse_{bucket}.log'.replace('/', '_')
  cmd=['gcsfuse','--implicit-dirs','--key-file', key,'--temp-dir','/content/gcsfuse_tmp','--log-file',log, bucket, mnt]
  res=subprocess.run(cmd, capture_output=True, text=True)
  ok=(res.returncode==0)
  print(f'[mount] {bucket} -> {mnt}:', ok)
  if not ok:
    print('[mount] stdout:\n' + res.stdout)
    print('[mount] stderr:\n' + res.stderr)
    try:
      tail=subprocess.run(['bash','-lc', f'tail -n 80 {log}'], capture_output=True, text=True)
      if tail.stdout: print('[mount] log tail:\n' + tail.stdout)
    except Exception: pass
  return ok
subprocess.run(['fusermount','-u','/content/src_gcs'], check=False)
subprocess.run(['fusermount','-u','/content/artifacts'], check=False)
# Quick bucket existence check
for b in ('pik_source_bucket','pik-artifacts-dev'):
  subprocess.run(['bash','-lc', f'gsutil ls -b gs://{b} || true'], check=False)
ok1 = mount('pik_source_bucket','/content/src_gcs')
ok2 = mount('pik-artifacts-dev','/content/artifacts')
print('mount src=', ok1, ' mount artifacts=', ok2)
if not (ok1 and ok2):
  print(textwrap.dedent('''
    [hint] If mount keeps failing:
      - Check SA has Storage Object Admin on both buckets
      - Try fallback: copy files with gsutil (already used elsewhere in the notebook)
      - Ensure bucket names are correct and exist (see checks above)
    '''))


Using SA key: /content/Secrets/sa.json
[mount] pik_source_bucket -> /content/src_gcs: True
[mount] pik-artifacts-dev -> /content/artifacts: True
mount src= True  mount artifacts= True


In [9]:
#@title Install Torch + SAM/SAM2 + GroundedDINO (CUDA-aware)
require_start()

# 1) Update base tooling
!pip -q install --upgrade pip setuptools wheel
!pip -q install -U jedi>=0.16 typing_extensions>=4.14.0 filelock>=3.15

# 2) Torch for detected CUDA (12.4 or 12.1); fallback to CPU
import subprocess, re
def _probe_cuda_tag():
    try:
        out = subprocess.check_output(['bash','-lc','nvcc --version || cat /usr/local/cuda/version.json || true'], text=True)
        m = re.search(r'release (\d+)\.(\d+)', out) or re.search(r'"cuda":\s*"(\d+)\.(\d+)"', out)
        if m:
            major, minor = m.groups()
            ver = f"{major}.{minor}"
        else:
            ver = None
    except Exception:
        ver = None
    if ver and ver.startswith('12.4'):
        return 'cu124'
    if ver and ver.startswith('12.1'):
        return 'cu121'
    # Default to cu121 in Colab
    return 'cu121'

_tag = _probe_cuda_tag()
print('[torch] installing for', _tag)
try:
    get_ipython().system("pip -q install --upgrade --force-reinstall torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/%s" % _tag)
except Exception as e:
    print('[warn] Torch install failed for', _tag, 'falling back to CPU:', e)
    get_ipython().system("pip -q install --upgrade --force-reinstall torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu")

# 3) Core deps
!pip -q install 'numpy<2.1,>=1.24'
!pip -q install shapely timm opencv-python pycocotools addict yacs requests pillow huggingface_hub

# 4) SAM and SAM2
!pip -q install git+https://github.com/facebookresearch/segment-anything.git
!pip -q install git+https://github.com/facebookresearch/segment-anything-2.git

# 5) GroundingDINO — pip first, fallback to source build
import sys, os, subprocess, importlib

print('[GroundingDINO] installing via pip…')
!pip -q install "git+https://github.com/IDEA-Research/GroundingDINO.git"

try:
    importlib.import_module('groundingdino')
    print('✅ GroundingDINO pip import OK')
except ImportError:
    print('⚠️ pip install failed, building from source…')
    if '/content/GroundingDINO' not in sys.path:
        sys.path.append('/content/GroundingDINO')
    !rm -rf /content/GroundingDINO
    !git clone --depth 1 https://github.com/IDEA-Research/GroundingDINO.git /content/GroundingDINO
    # Build C++ extensions
    try:
        subprocess.check_call(['bash','-lc','sudo apt-get -q update && sudo apt-get -q install -y ninja-build'])
    except Exception as e:
        print('[warn] apt-get ninja-build failed:', e)
    build_dir = '/content/GroundingDINO'
    orig = os.getcwd()
    os.chdir(build_dir)
    try:
        res = subprocess.run([sys.executable, 'setup.py', 'build_ext', '--inplace'], capture_output=True, text=True)
        if res.returncode != 0:
            print('🟥 build_ext failed:
', res.stderr)
        else:
            print('✅ build_ext ok')
    finally:
        os.chdir(orig)
    try:
        if 'groundingdino' in sys.modules:
            importlib.reload(sys.modules['groundingdino'])
        else:
            importlib.import_module('groundingdino')
        print('✅ GroundingDINO import OK after source build')
    except Exception as e:
        print('🟥 GroundingDINO import still failing:', e)

# 6) Final checks
try:
    import torch, numpy
    print(f"[versions] torch: {torch.__version__}, numpy: {numpy.__version__}")
    from groundingdino.util.inference import Model
    print('✅ GroundingDINO.Model available')
    import segment_anything as _sam
    print('✅ SAM v1 import OK')
    try:
        import sam2 as _sam2
        print('✅ SAM2 import OK')
    except Exception as e:
        print('[warn] SAM2 import failed:', e)
    print('CUDA available:', torch.cuda.is_available())
except Exception as e:
    print('🟥 Final checks failed:', e)

# 7) Harmonize
!pip -q uninstall -y xformers || true
!pip -q install -U 'numpy<2.1,>=1.24' typing_extensions>=4.14.0 filelock>=3.15 gcsfs==2025.3.0 fsspec==2025.3.0


[torch] installing for cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.4 requires typing_extensions>=4.14.0, but you have typing-extensions 4.12.2 which is incompatible.
pytensor 2.31.7 requires filelock>=3.15, but you have filelock 3.13.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.31.7 requires filelock>=3.15, but you have filelock 3.13.1 which is incompatible.
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing

In [10]:
#@title CUDA and C++ ops self-check
import warnings, importlib, torch, torchvision, os
warnings.filterwarnings('ignore', category=FutureWarning)
req = (DEVICE.lower() if 'DEVICE' in globals() else 'auto')
cuda_avail = torch.cuda.is_available()
print('torch:', torch.__version__, 'cuda:', torch.version.cuda, 'available:', cuda_avail)
print('torchvision:', torchvision.__version__)
tv_ops_ok=False
if cuda_avail:
  try:
    x=torch.rand(256,4,device='cuda'); y=torch.rand(256,4,device='cuda')
    from torchvision.ops import box_iou
    _=box_iou(x,y); tv_ops_ok=True; print('[OK] torchvision.ops on CUDA')
  except Exception as e:
    print('[WARN] torchvision.ops CUDA failed:', e)
try:
  m=importlib.import_module('groundingdino.models.GroundingDINO.ms_deform_attn')
  dino_ops_ok=bool(getattr(m,'_C', None))
  print('GroundingDINO _C present:', dino_ops_ok)
except Exception as e:
  dino_ops_ok=False; print('[WARN] GroundingDINO C++ ops import failed:', e)
# Hard assertions when DEVICE='cuda'
if req=='cuda':
  assert cuda_avail, 'CUDA requested but not available'
  assert tv_ops_ok, 'torchvision CUDA ops unavailable'
  assert dino_ops_ok, 'GroundingDINO C++ ops (_C) not built'
print('[SELF-CHECK] req=', req, ' cuda_avail=', cuda_avail, ' tv_ops_ok=', tv_ops_ok, ' dino_ops_ok=', dino_ops_ok)


torch: 2.5.1+cu121 cuda: 12.1 available: True
torchvision: 0.20.1+cu121
[OK] torchvision.ops on CUDA
GroundingDINO _C present: True
[SELF-CHECK] req= auto  cuda_avail= True  tv_ops_ok= True  dino_ops_ok= True


In [11]:
#@title Download/Resolve Model Weights
#@title Download/Resolve Model Weights
# (Optional) Download model weights if not present
import os, pathlib, shutil, subprocess
from typing import Optional
pathlib.Path('/content/models/groundingdino').mkdir(parents=True, exist_ok=True)
pathlib.Path('/content/models/sam').mkdir(parents=True, exist_ok=True)
GROUNDING_MODEL = '/content/models/groundingdino/groundingdino_swint_ogc.pth'
SAM_MODEL = '/content/models/sam/sam_vit_h_4b8939.pth'
GROUNDING_URL = 'https://github.com/IDEA-Research/GroundingDINO/releases/download/0.1.0/groundingdino_swint_ogc.pth'
SAM_URL = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth'
SAM2_MODEL = '/content/models/sam2/sam2_hiera_large.pt'
SAM2_URL = 'https://huggingface.co/facebook/sam2-hiera-large/resolve/main/sam2_hiera_large.pt'
# Try to read HF token from Colab Keys or env
HF_TOKEN = os.getenv('HF_TOKEN', '')
try:
  from google.colab import userdata as _ud
  HF_TOKEN = _ud.get('HF_TOKEN') or HF_TOKEN
except Exception:
  pass
def _file_ok(p: str, min_size: int) -> bool:
  try:
    return os.path.exists(p) and os.path.getsize(p) >= min_size
  except Exception:
    return False
def _try_torch_load(p: str) -> bool:
  try:
    import torch
    torch.load(p, map_location='cpu')
    return True
  except Exception as e:
    print('[warn] torch.load failed:', e)
    return False
def _hf_download(repo_id: str, filename: str, dest: str) -> bool:
  try:
    from huggingface_hub import hf_hub_download, login
    if HF_TOKEN:
      try:
        login(token=HF_TOKEN)
      except Exception as e:
        print('[warn] HF login failed:', e)
    ckpt = hf_hub_download(repo_id=repo_id, filename=filename, local_dir=os.path.dirname(dest), local_dir_use_symlinks=False, token=HF_TOKEN or None)
    if ckpt != dest:
      shutil.copy2(ckpt, dest)
    return True
  except Exception as e:
    print('[warn] HF download failed:', e)
    return False
def _curl(url: str, dest: str, min_size: int) -> bool:
  cmd = f"curl -L --fail --retry 5 --retry-all-errors -o '{dest}.tmp' '{url}'"
  rc = subprocess.call(cmd, shell=True)
  if rc == 0 and _file_ok(dest + '.tmp', min_size):
    shutil.move(dest + '.tmp', dest)
    return True
  else:
    print('[warn] curl download insufficient or failed:', rc)
    try:
      os.remove(dest + '.tmp')
    except Exception:
      pass
    return False
# GroundedDINO (expect ~0.9GB)
MIN_DINO = 600_000_000
need_dino = (not _file_ok(GROUNDING_MODEL, MIN_DINO)) or (not _try_torch_load(GROUNDING_MODEL))
if need_dino:
  print('Downloading GroundingDINO weights (robust)...')
  try:
    os.remove(GROUNDING_MODEL)
  except Exception:
    pass
  # 1) Try GCS mirror if mounted
  gcs_mirror = '/content/artifacts/models/groundingdino/groundingdino_swint_ogc.pth'
  ok = _file_ok(gcs_mirror, MIN_DINO)
  if ok:
    try:
      shutil.copy2(gcs_mirror, GROUNDING_MODEL)
      print('[DINO] using GCS mirror')
      ok = True
    except Exception as e:
      print('[warn] copy from GCS mirror failed:', e); ok = False
  # 2) Try HF Hub (public repo)
  if (not ok):
    ok = _hf_download('ShilongLiu/GroundingDINO', 'groundingdino_swint_ogc.pth', GROUNDING_MODEL)
    if ok: print('[DINO] using HF Hub')
  # 3) Try GitHub release via curl
  if (not ok) or (not _file_ok(GROUNDING_MODEL, MIN_DINO)):
    ok = _curl(GROUNDING_URL, GROUNDING_MODEL, MIN_DINO)
    if ok: print('[DINO] using curl URL')
  # 4) Try gsutil from bucket path if accessible
  if (not ok) or (not _file_ok(GROUNDING_MODEL, MIN_DINO)):
    try:
      rc = subprocess.call(f"gsutil cp gs://pik-artifacts-dev/models/groundingdino/groundingdino_swint_ogc.pth '{GROUNDING_MODEL}'", shell=True)
      ok = (rc == 0) and _file_ok(GROUNDING_MODEL, MIN_DINO)
      if ok: print('[DINO] using gsutil mirror')
    except Exception as e:
      print('[warn] gsutil mirror copy failed:', e)
  if (not ok) or (not _file_ok(GROUNDING_MODEL, MIN_DINO)) or (not _try_torch_load(GROUNDING_MODEL)):
    raise SystemExit('Failed to fetch a valid GroundingDINO checkpoint')
# SAM (ViT-H is large; check size only)
MIN_SAM = 1_000_000_000
if not _file_ok(SAM_MODEL, MIN_SAM):
  print('Downloading SAM ViT-H weights (robust)...')
  # 1) Try GCS mirror if mounted
  sam_gcs_mirror = '/content/artifacts/models/sam/sam_vit_h_4b8939.pth'
  ok = _file_ok(sam_gcs_mirror, MIN_SAM)
  if ok:
    try:
      shutil.copy2(sam_gcs_mirror, SAM_MODEL)
      print('[SAM] using GCS mirror')
      ok = True
    except Exception as e:
      print('[warn] copy SAM from GCS mirror failed:', e); ok = False
  # 2) Try HF Hub
  if (not ok):
    ok = _hf_download('facebook/sam', 'sam_vit_h_4b8939.pth', SAM_MODEL)
    if ok: print('[SAM] using HF Hub')
  # 3) Try official URL via curl
  if (not ok) or (not _file_ok(SAM_MODEL, MIN_SAM)):
    ok = _curl(SAM_URL, SAM_MODEL, MIN_SAM)
    if ok: print('[SAM] using curl URL')
  # 4) Try gsutil mirror from bucket
  if (not ok) or (not _file_ok(SAM_MODEL, MIN_SAM)):
    try:
      rc = subprocess.call(f"gsutil cp gs://pik-artifacts-dev/models/sam/sam_vit_h_4b8939.pth '{SAM_MODEL}'", shell=True)
      ok = (rc == 0) and _file_ok(SAM_MODEL, MIN_SAM)
      if ok: print('[SAM] using gsutil mirror')
    except Exception as e:
      print('[warn] gsutil SAM mirror copy failed:', e)
  if (not ok) or (not _file_ok(SAM_MODEL, MIN_SAM)):
    raise SystemExit('Failed to fetch SAM ViT-H checkpoint')
# SAM2 (Hiera Large)
MIN_SAM2 = 700_000_000
if not _file_ok(SAM2_MODEL, MIN_SAM2):
  print('Downloading SAM2 Hiera Large weights (robust)...')
  # 1) Try GCS mirror if mounted
  sam2_gcs_mirror = '/content/artifacts/models/sam2/sam2_hiera_large.pt'
  ok = _file_ok(sam2_gcs_mirror, MIN_SAM2)
  if ok:
    try:
      shutil.copy2(sam2_gcs_mirror, SAM2_MODEL)
      print('[SAM2] using GCS mirror')
      ok = True
    except Exception as e:
      print('[warn] copy SAM2 from GCS mirror failed:', e); ok = False
  # 2) Try HF Hub
  if (not ok):
    ok = _hf_download('facebook/sam2-hiera-large', 'sam2_hiera_large.pt', SAM2_MODEL)
    if ok: print('[SAM2] using HF Hub')
  # 3) Try direct URL via curl
  if (not ok) or (not _file_ok(SAM2_MODEL, MIN_SAM2)):
    ok = _curl(SAM2_URL, SAM2_MODEL, MIN_SAM2)
    if ok: print('[SAM2] using curl URL')
  # 4) Try gsutil mirror from bucket
  if (not ok) or (not _file_ok(SAM2_MODEL, MIN_SAM2)):
    try:
      rc = subprocess.call(f"gsutil cp gs://pik-artifacts-dev/models/sam2/sam2_hiera_large.pt '{SAM2_MODEL}'", shell=True)
      ok = (rc == 0) and _file_ok(SAM2_MODEL, MIN_SAM2)
      if ok: print('[SAM2] using gsutil mirror')
    except Exception as e:
      print('[warn] gsutil SAM2 mirror copy failed:', e)
  if (not ok) or (not _file_ok(SAM2_MODEL, MIN_SAM2)):
    raise SystemExit('Failed to fetch SAM2 Hiera Large checkpoint')
import os as _os; print('[DINO] size=', _os.path.getsize(GROUNDING_MODEL)); print('GROUNDING_MODEL =', GROUNDING_MODEL)
import os as _os; print('[SAM2] size=', _os.path.getsize(SAM2_MODEL))
import os as _os; print('[SAM] size=', _os.path.getsize(SAM_MODEL)); print('SAM_MODEL       =', SAM_MODEL)

# Log weights info if logging enabled
try:
  import os as _os
  WEIGHTS_INFO = {
    'groundingdino': {'path': GROUNDING_MODEL, 'size': _os.path.getsize(GROUNDING_MODEL)},
    'sam': {'path': SAM_MODEL, 'size': _os.path.getsize(SAM_MODEL)},
    'sam2': ({'path': SAM2_MODEL, 'size': _os.path.getsize(SAM2_MODEL)} if _os.path.exists(SAM2_MODEL) else None),
  }
  if 'log_json' in globals(): log_json('weights.json', WEIGHTS_INFO)
except Exception as e:
  print('[LOG] weights info not recorded:', e)


[DINO] using GCS mirror
[SAM] using GCS mirror
[warn] copy SAM2 from GCS mirror failed: [Errno 2] No such file or directory: '/content/models/sam2/sam2_hiera_large.pt'


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.


sam2_hiera_large.pt:   0%|          | 0.00/898M [00:00<?, ?B/s]

[SAM2] using HF Hub
[DINO] size= 693997677
GROUNDING_MODEL = /content/models/groundingdino/groundingdino_swint_ogc.pth
[SAM2] size= 897952466
[SAM] size= 2564550879
SAM_MODEL       = /content/models/sam/sam_vit_h_4b8939.pth


In [40]:
#@title Logging helpers (GCS)
import os, sys, json, time, platform, socket, subprocess
from pathlib import Path
RUN_ID = time.strftime('%Y%m%d-%H%M%S') + (('-'+RUN_TAG.strip()) if ('RUN_TAG' in globals() and RUN_TAG.strip()) else '')
LOCAL_LOG_ROOT = '/content/artifacts/colab_runs' if os.path.exists('/content/artifacts') else '/content/colab_runs'
LOG_DIR = Path(LOCAL_LOG_ROOT)/RUN_ID
LOG_DIR.mkdir(parents=True, exist_ok=True)

OUT_PAGES_DIR = '/content/pages'
PLAYBOOK_PDF = '/content/Playbook.pdf'
DETECT_OUT = '/content/grounded_regions'

def log_json(name, obj):
  p = LOG_DIR/name if isinstance(name, Path) else LOG_DIR/str(name)
  p.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding='utf-8')
def log_kv(key, val):
  data = {}
  p = LOG_DIR/'run.json'
  if p.exists():
    try: data = json.loads(p.read_text(encoding='utf-8'))
    except Exception: data = {}
  data[key]=val
  p.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding='utf-8')
env = { 'python': sys.version, 'platform': platform.platform(), 'hostname': socket.gethostname(), 'device_req': (DEVICE if 'DEVICE' in globals() else 'auto') }
try:
  import torch
  env.update({'torch': torch.__version__, 'cuda': getattr(torch.version,'cuda',None), 'cuda_available': torch.cuda.is_available()})
except Exception: pass
log_json('env.json', env)
def upload_logs():
  if not ('REPORT_TO_GCS' in globals() and REPORT_TO_GCS):
    print('[LOG] REPORT_TO_GCS disabled'); return
  bucket = (GCS_BUCKET if 'GCS_BUCKET' in globals() else 'pik-artifacts-dev')
  prefix = f'colab_runs/{RUN_ID}'
  try:
    from google.cloud import storage
    client=storage.Client()
    b=client.bucket(bucket)
    for p in LOG_DIR.rglob('*'):
      if p.is_file():
        rel=str(p.relative_to(LOG_DIR)).replace('\\','/')
        blob=b.blob(f'{prefix}/{rel}')
        ctype='application/json' if p.suffix.lower()=='.json' else 'text/plain; charset=utf-8'
        blob.content_type=ctype
        blob.upload_from_filename(str(p))
    print(f'[LOG] uploaded to gs://{bucket}/{prefix}')
    return
  except Exception as e:
    print('[LOG] storage client failed, fallback to gsutil:', e)
  # gsutil fallback
  cmd=f"gsutil -m cp -r '{LOG_DIR}' gs://{bucket}/colab_runs/"
  subprocess.run(['bash','-lc', cmd], check=False)

In [41]:
#@title Source Full Run — playbooks (PDF) + frames (PDF+PNG) из /content/src_gcs
# Источники (смонтированные бакеты):
#   /content/src_gcs/playbooks   (PDF, рекурсивно **/*.pdf)
#   /content/src_gcs/frames      (PDF+PNG, рекурсивно **/*.(pdf|png))
# Выход:
#   /content/pages_src/playbooks/<doc>/page-<n>.png
#   /content/pages_src/frames/<doc>/page-<n>.png
#   /content/full_run_manifest.jsonl        ({"image": "<abs path>"}, без дублей)

import os, re, glob, json, subprocess, time
from pathlib import Path

# Смонтированный источник
MOUNT_ROOT     = '/content/src_gcs'                # pik_source_bucket
PLAYBOOKS_DIR  = f'{MOUNT_ROOT}/playbooks'         # PDF
FRAMES_DIR     = f'{MOUNT_ROOT}/frames'            # PDF + PNG

# Выходные пути
OUT_PAGES_ROOT = '/content/pages_src'
OUT_PAGES_PB   = f'{OUT_PAGES_ROOT}/playbooks'
OUT_PAGES_FR   = f'{OUT_PAGES_ROOT}/frames'
MANIFEST       = '/content/full_run_manifest.jsonl'

# Параметры рендера
DPI = 150
RENDER_EXISTING = False        # перерисовывать существующие page-*.png?
MAX_PAGES_PER_DOC = 0          # 0 = все страницы

# Предусловия: playbooks/frames должны существовать
for p in (PLAYBOOKS_DIR, FRAMES_DIR):
    if not Path(p).exists():
        raise SystemExit(f"[fatal] Не найден путь: {p}. Проверь монтирование GCS.")

# Проверка poppler (pdftoppm/pdfinfo)
def _ensure_poppler():
    from shutil import which
    missing = [x for x in ('pdftoppm','pdfinfo') if which(x) is None]
    if missing:
        raise SystemExit(f"[fatal] Не найдены утилиты {missing}. Установи poppler-utils: !apt-get -q update && apt-get -q install -y poppler-utils")
_ensure_poppler()

def _pdf_pages(pdf_path: str) -> int:
    try:
        out = subprocess.check_output(['pdfinfo', pdf_path], text=True)
        m = re.search(r'^Pages:\s+(\d+)', out, re.M)
        return int(m.group(1)) if m else 0
    except Exception:
        return 0

def _render_doc(pdf: str, out_root: str, dpi: int = 150, limit: int = 0, rerender: bool = False) -> int:
    """Рендер одного PDF в out_root/<doc>/page-*.png. Возвращает кол-во страниц (написанных или пропущенных)."""
    name = Path(pdf).stem
    out_dir = Path(out_root) / name
    out_dir.mkdir(parents=True, exist_ok=True)
    n = _pdf_pages(pdf)
    if n <= 0:
        print(f"[warn] Не удалось определить число страниц: {pdf}. Пропуск.")
        return 0
    if limit and limit > 0:
        n = min(n, limit)
    print(f"[render] {name}: pages={n} (dpi={dpi})")

    done = 0
    for p in range(1, n + 1):
        png = out_dir / f'page-{p}.png'
        if png.exists() and not rerender:
            done += 1
            continue
        cmd = ['pdftoppm','-f',str(p),'-l',str(p),'-png','-singlefile','-r',str(dpi), pdf, str(png.with_suffix(''))]
        subprocess.run(cmd, check=True)
        done += 1
        if (done % 10) == 0:
            print(f"  .. {done}/{n}")
    return done

def _load_manifest_set(path: str) -> set:
    s = set()
    mp = Path(path)
    if mp.exists():
        with mp.open('r', encoding='utf-8') as f:
            for line in f:
                line=line.strip()
                if not line: continue
                try:
                    s.add(json.loads(line)['image'])
                except Exception:
                    pass
    return s

# Основной поток
Path(OUT_PAGES_PB).mkdir(parents=True, exist_ok=True)
Path(OUT_PAGES_FR).mkdir(parents=True, exist_ok=True)
t0 = time.time()

# 1) PDF: playbooks → render
pdfs_pb = sorted(glob.glob(os.path.join(PLAYBOOKS_DIR, '**', '*.pdf'), recursive=True))
print('[scan] Playbooks PDF :', len(pdfs_pb))
pages_written_pb = 0
for i, pdf in enumerate(pdfs_pb, start=1):
    try:
        pages_written_pb += _render_doc(pdf, OUT_PAGES_PB, dpi=DPI, limit=MAX_PAGES_PER_DOC, rerender=RENDER_EXISTING)
    except Exception as e:
        print(f"[error] playbooks pdf fail {pdf}:", e)
    if (i % 5) == 0:
        print(f"[progress] playbooks {i}/{len(pdfs_pb)}")

# 2) PDF: frames → render
pdfs_fr = sorted(glob.glob(os.path.join(FRAMES_DIR, '**', '*.pdf'), recursive=True))
print('[scan] Frames PDF    :', len(pdfs_fr))
pages_written_fr = 0
for i, pdf in enumerate(pdfs_fr, start=1):
    try:
        pages_written_fr += _render_doc(pdf, OUT_PAGES_FR, dpi=DPI, limit=MAX_PAGES_PER_DOC, rerender=RENDER_EXISTING)
    except Exception as e:
        print(f"[error] frames pdf fail {pdf}:", e)
    if (i % 5) == 0:
        print(f"[progress] frames-pdf {i}/{len(pdfs_fr)}")

# 3) PNG: frames → просто собрать
frames_png = sorted(glob.glob(os.path.join(FRAMES_DIR, '**', '*.png'), recursive=True))
print('[scan] Frames PNG    :', len(frames_png))

# 4) Обновить манифест (без дублей)
existing = _load_manifest_set(MANIFEST)
added = 0
with Path(MANIFEST).open('a', encoding='utf-8') as f:
    # Страницы из playbooks
    page_pngs_pb = sorted(glob.glob(os.path.join(OUT_PAGES_PB, '**', 'page-*.png'), recursive=True))
    for im in page_pngs_pb:
        if im not in existing:
            f.write(json.dumps({'image': im}, ensure_ascii=False) + '\n')
            existing.add(im); added += 1
    # Страницы из frames (PDF)
    page_pngs_fr = sorted(glob.glob(os.path.join(OUT_PAGES_FR, '**', 'page-*.png'), recursive=True))
    for im in page_pngs_fr:
        if im not in existing:
            f.write(json.dumps({'image': im}, ensure_ascii=False) + '\n')
            existing.add(im); added += 1
    # Исходные кадры PNG
    for im in frames_png:
        if im not in existing:
            f.write(json.dumps({'image': im}, ensure_ascii=False) + '\n')
            existing.add(im); added += 1

t1 = time.time()
print('---- Summary ----')
print('Playbooks PDF                    :', len(pdfs_pb))
print('Frames PDF                       :', len(pdfs_fr))
print('Frames PNG                       :', len(frames_png))
print('Pages rendered (playbooks+frames):', pages_written_pb + pages_written_fr)
print('Manifest appended                :', added)
print('Manifest                         :', MANIFEST)
print(f'Elapsed                          : {t1 - t0:.1f}s')


[scan] Playbooks PDF : 3
[render] 2023-06 - fastbreakOne - Expert Guide - Ecosystem Strategy  - English: pages=62 (dpi=150)
[render] PIK - Expert Guide - Platform IT Architecture - Playbook - v11: pages=50 (dpi=150)
[render] PIK 5-0 - Introduction - English: pages=49 (dpi=150)
[scan] Frames PDF    : 29
[render] L1 - Ecosystems Portfolio Map - v01: pages=1 (dpi=150)
[render] L2 - Ecosystem Strategy Map: pages=1 (dpi=150)
[render] L2 - Market Ecosystem Journey: pages=1 (dpi=150)
[render] L3 - Platform Value Network Canvas: pages=1 (dpi=150)
[render] PIK - Expert Guide - Platform IT Architecture - Assessment - v01: pages=1 (dpi=150)
[progress] frames-pdf 5/29
[render] PIK - Platform IT Architecture Canvas - Table View - v01: pages=1 (dpi=150)
[render] PIK - Platform IT Architecture Canvases - v01: pages=1 (dpi=150)
[render] PIK 5-0 - Ecosystem Forces Scan - ENG: pages=1 (dpi=150)
[render] PIK 5-0 - Longtail Discovery - ENG: pages=1 (dpi=150)
[render] PIK 5-0 - MVP - ENG: pages=1 (dpi=150)

In [42]:
# Подготовить список изображений для Боевого режима из манифеста
import json
MANIFEST = '/content/full_run_manifest.jsonl'
images = [json.loads(l)['image'] for l in open(MANIFEST, 'r', encoding='utf-8') if l.strip()]
IMAGES = images  # если боевой режим ждёт именно IMAGES
print('Images to process:', len(images))


Images to process: 376


In [45]:
#@title Define detect_image_to_regions (SAM/SAM2 — real regions)
import os, json, base64, io
import numpy as np
from pathlib import Path
from PIL import Image

# Параметры весов
SAM2_MODEL = os.environ.get('SAM2_MODEL', '/content/models/sam2/sam2_hiera_large.pt')
SAM_V1_CANDIDATES = [
    os.environ.get('SAM_MODEL', ''),                     # приоритет — если задан
    '/content/models/sam/sam_vit_b_01ec64.pth',          # компактный (~370MB)
    '/content/models/sam/sam_vit_l_0b3195.pth',
    '/content/models/sam/sam_vit_h_4b8939.pth',
]
AUTO_DOWNLOAD_SAM_V1 = False  # поставь True, если нет весов SAM v1, и можно подождать скачивание (~370MB для ViT-B)

def _b64_png(img: Image.Image) -> str:
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')

def _bbox_from_mask(mask: np.ndarray) -> tuple:
    # mask: HxW boolean/0-1
    ys, xs = np.where(mask > 0)
    if len(xs) == 0 or len(ys) == 0:
        return (0, 0, -1, -1)
    x0, x1 = int(xs.min()), int(xs.max())
    y0, y1 = int(ys.min()), int(ys.max())
    return (x0, y0, x1 - x0 + 1, y1 - y0 + 1)

def _try_sam2():
    try:
        import torch
        from sam2.build_sam import build_sam2
        from sam2.sam2_image_predictor import SAM2ImagePredictor
        if not (SAM2_MODEL and Path(SAM2_MODEL).exists()):
            return None
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = build_sam2('sam2_hiera_large', SAM2_MODEL, device=device)
        predictor = SAM2ImagePredictor(model)
        return ('sam2', predictor, device)
    except Exception as e:
        print('[warn] SAM2 init failed:', e)
        return None

def _try_sam_v1():
    try:
        import torch
        from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
        ckpt = next((p for p in SAM_V1_CANDIDATES if p and Path(p).exists()), None)
        if ckpt is None and AUTO_DOWNLOAD_SAM_V1:
            # Скачиваем лёгкий ViT-B
            url = 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth'
            Path('/content/models/sam').mkdir(parents=True, exist_ok=True)
            dst = '/content/models/sam/sam_vit_b_01ec64.pth'
            import subprocess
            print('[sam] downloading:', url)
            subprocess.run(['bash','-lc', f"curl -L '{url}' -o '{dst}'"], check=True)
            ckpt = dst
        if ckpt is None:
            return None
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        arch = 'vit_b' if 'vit_b' in ckpt else ('vit_l' if 'vit_l' in ckpt else 'vit_h')
        sam = sam_model_registry[arch](checkpoint=ckpt).to(device)
        mask_gen = SamAutomaticMaskGenerator(
            sam,
            points_per_side=16,
            pred_iou_thresh=0.88,
            stability_score_thresh=0.92,
            min_mask_region_area=4000,
        )
        return ('sam_v1', mask_gen, device)
    except Exception as e:
        print('[warn] SAM v1 init failed:', e)
        return None

# Ленивая инициализация (один раз на сессию)
_DET_BACKEND = {'kind': None, 'obj': None, 'device': None}

def _ensure_backend():
    if _DET_BACKEND['kind'] is not None:
        return _DET_BACKEND
    b = _try_sam2()
    if b is None:
        b = _try_sam_v1()
    if b is None:
        raise SystemExit('[fatal] No SAM/SAM2 backend available. Provide weights or enable AUTO_DOWNLOAD_SAM_V1=True.')
    _DET_BACKEND['kind'], _DET_BACKEND['obj'], _DET_BACKEND['device'] = b
    print('[detector] using', _DET_BACKEND['kind'], 'on', _DET_BACKEND['device'])
    return _DET_BACKEND

def detect_image_to_regions(image_path: str, out_dir: str, max_regions: int = 8):
    """
    Реальная детекция регионов:
    - если доступен SAM2 — используем его (grid‑prompt + bbox из масок);
    - иначе SAM v1 (Automatic Mask Generator).
    Сохраняет region-*.json (+ preview region-*.png) в out_dir/regions.
    """
    be = _ensure_backend()
    kind, obj, device = be['kind'], be['obj'], be['device']

    img = Image.open(image_path).convert('RGB')
    img_np = np.array(img)

    regions_dir = Path(out_dir) / 'regions'
    regions_dir.mkdir(parents=True, exist_ok=True)

    masks = []
    if kind == 'sam2':
        # Простейшая сетка точек для инициализации (быстро и надёжно)
        from sam2.sam2_image_predictor import SAM2ImagePredictor
        obj.set_image(img_np)
        H, W = img_np.shape[:2]
        steps = 5
        xs = np.linspace(W*0.1, W*0.9, steps).astype(int)
        ys = np.linspace(H*0.1, H*0.9, steps).astype(int)
        for y in ys:
            for x in xs:
                masks_pred, scores, _ = obj.predict(point_coords=np.array([[x,y]]), point_labels=np.array([1]), multimask_output=True)
                # masks_pred: (k, H, W) <- boolean
                for m in masks_pred:
                    masks.append(m.astype(np.uint8))
    else:
        # SAM v1 — автоматический генератор масок
        masks_gen = obj.generate(img_np)  # list of dicts: {'segmentation', 'bbox', ...}
        for m in masks_gen:
            seg = m.get('segmentation')
            if seg is not None:
                masks.append(seg.astype(np.uint8))

    # Отсортируем по площади и возьмём топ‑N
    scored = []
    for m in masks:
        area = int((m > 0).sum())
        if area > 1000:
            scored.append((area, m))
    scored.sort(reverse=True)
    picked = [m for _, m in scored[:max_regions]]

    # Запись регионов
    written = 0
    for i, m in enumerate(picked, start=1):
        x, y, w, h = _bbox_from_mask(m)
        if w <= 0 or h <= 0:
            continue
        crop = img.crop((x, y, x + w, y + h))
        b64 = _b64_png(crop)
        (regions_dir / f'region-{i}.json').write_text(
            json.dumps({'bbox': {'x': x, 'y': y, 'w': w, 'h': h}, 'text': '', 'image_b64': b64}, ensure_ascii=False),
            encoding='utf-8'
        )
        crop.save(regions_dir / f'region-{i}.png', 'PNG')
        written += 1

    if written == 0:
        # В крайнем случае — хотя бы один регион (всё изображение)
        x, y, w, h = 0, 0, img.width, img.height
        b64 = _b64_png(img)
        (regions_dir / f'region-1.json').write_text(
            json.dumps({'bbox': {'x': x, 'y': y, 'w': w, 'h': h}, 'text': '', 'image_b64': b64}, ensure_ascii=False),
            encoding='utf-8'
        )
        img.save(regions_dir / f'region-1.png', 'PNG')
        written = 1

    print(f"[detect] {Path(image_path).name}: regions={written} → {regions_dir}")


In [44]:
# Боевой режим: GroundedDINO → SAM
require_start()

import os, json, pathlib, cv2, numpy as np, torch
from groundingdino.util.inference import Model
# SAM/SAM2 init with fallback and device control
_req = (DEVICE.lower() if 'DEVICE' in globals() else 'auto')
if _req == 'cuda' and not torch.cuda.is_available():
  print('[warn] CUDA requested but not available; using CPU')
  device = 'cpu'
elif _req == 'cpu':
  device = 'cpu'
else:
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Selected device:', device)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam2_predictor = None
if 'USE_SAM2' in globals() and USE_SAM2:
  try:
    from sam2.build_sam import build_sam2
    from sam2.sam2_image_predictor import SAM2ImagePredictor
    sam2_model = build_sam2('sam2_hiera_large', SAM2_MODEL, device=device)
    sam2_predictor = SAM2ImagePredictor(sam2_model)
    print('SAM2 ready on', device)
  except Exception as e:
    print('SAM2 init failed, fallback to SAM v1:', e)
    sam2_predictor = None
if sam2_predictor is None:
  from segment_anything import sam_model_registry, SamPredictor
  print('SAM v1 ready on', device)
CFG_PATH = '/content/GroundingDINO_SwinT_OGC.py'
CFG_URL = 'https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinT_OGC.py'
# Попытка скачать конфиг, если его нет
import urllib.request, urllib.error
def _download(url, path):
  try:
    urllib.request.urlretrieve(url, path)
    return os.path.exists(path) and os.path.getsize(path) > 1000
  except Exception:
    return False
if not os.path.exists(CFG_PATH):
  ok = _download(CFG_URL, CFG_PATH)
  if not ok:
    try:
      import groundingdino, os as _os
      CFG_PATH = _os.path.join(_os.path.dirname(groundingdino.__file__), 'config', 'GroundingDINO_SwinT_OGC.py')
      print('Using package config at', CFG_PATH)
    except Exception as e:
      raise FileNotFoundError('GroundingDINO config not found and download failed')
# Sanity check on DINO checkpoint
import torch
try:
  _ = torch.load(GROUNDING_MODEL, map_location='cpu')
except Exception as e:
  raise RuntimeError(f'GroundedDINO checkpoint invalid: {e}')
gd_model = Model(model_config_path=CFG_PATH, model_checkpoint_path=GROUNDING_MODEL, device=device)
def save_region(rdir, idx, img, xyxy):
  x0,y0,x1,y1 = map(int, xyxy)
  x0,y0 = max(0,x0), max(0,y0)
  crop = img[y0:y1, x0:x1] if y1>y0 and x1>x0 else img
  ok, buf = cv2.imencode('.png', crop)
  if ok: (rdir/f'region-{idx}.png').write_bytes(buf.tobytes())
  obj = { 'bbox': {'x':int(x0),'y':int(y0),'w':int(x1-x0),'h':int(y1-y0)}, 'text':'', 'image_b64':'' }
  (rdir/f'region-{idx}.json').write_text(json.dumps(obj, ensure_ascii=False), encoding='utf-8')

def ensure_dir(d):
    pathlib.Path(d).mkdir(parents=True, exist_ok=True)

def detect_one(image_path, out_root):
  img = cv2.imread(image_path); assert img is not None, image_path
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  H,W = img_rgb.shape[:2]
  output = gd_model.predict_with_classes(image=img_rgb, classes=PROMPTS, box_threshold=BOX_THRESHOLD, text_threshold=TEXT_THRESHOLD)
  print(f"Output from predict_with_classes: {output}") # Debugging line
  boxes = output.xyxy # Access boxes from the Detections object
  logits = output.confidence # Access confidence scores from the Detections object
  phrases = [PROMPTS[class_id] for class_id in output.class_id] # Infer phrases from class_id and PROMPTS
  bxs = []
  for b in boxes:
    b = np.asarray(b, dtype=float)
    if b.max()<=1.01: x0,y0,x1,y1 = b[0]*W, b[1]*H, b[2]*W, b[3]*H
    else: x0,y0,x1,y1 = b
    bxs.append([x0,y0,x1,y1])
  out = os.path.join(out_root, pathlib.Path(image_path).stem, 'regions'); ensure_dir(out)
  for i,xyxy in enumerate(bxs[:TOPK], start=1): save_region(pathlib.Path(out), i, img, xyxy)
# Страницы
ensure_dir(DETECT_OUT)
for p in PAGES:
  detect_one(f'/content/pages/page-{p}.png', DETECT_OUT)
# Фреймы
ensure_dir('/content/grounded_frames') # Ensure directory exists
for name in FRAME_NAMES:
  f = f'/content/src_gcs/frames/{name}'
  if os.path.exists(f): detect_one(f, '/content/grounded_frames')
# Выгрузка
!gsutil -m rsync -r /content/grounded_regions gs://pik-artifacts-dev/grounded_regions/
!gsutil -m rsync -r /content/grounded_frames gs://pik-artifacts-dev/grounded_regions/
!gsutil ls -r gs://pik-artifacts-dev/grounded_regions | head -n 40

Selected device: cuda
SAM v1 ready on cuda
final text_encoder_type: bert-base-uncased

both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
Starting synchronization...
Copying file:///content/grounded_regions/frames/PIK - Expert Guide - Platform IT Architecture - Assessment - v01/regions/region-1.json [Content-Type=application/json]...
Copying file:///content/grounded_regions/frames/PIK - Platform IT Architecture Canvas - Table View - v01/regions/region-1.json [Content-Type=application/json]...
Copying file:///content/grounded_regions/frames/PIK - Expert Guide - Platform IT Architecture - Assessment - v01/regions/region-1.png [Content-Type=image/png]...
Copying file:///content/grounded_regions/frames/PI

In [30]:
#@title Upload Regions to GCS (grounded_regions)
require_start()

# Синхронизация локальных регионов в GCS: либо через смонтированный бакет (/content/artifacts),
# либо через gsutil rsync. Идемпотентно.

import os, glob, subprocess, time
from pathlib import Path

# Локальный корень детекции (сюда пишет “боевой режим”)
LOCAL_REGIONS = '/content/grounded_regions'    # EXPECT: <unit>/regions/region-*.{json,png,...}
DEST_PREFIX   = 'grounded_regions'             # базовый префикс в bakete
RUN_TAG       = os.environ.get('RUN_TAG', '')  # например: '2025-09-16-1040' или 'full-run'
GCS_BUCKET    = os.environ.get('GCS_BUCKET', 'pik-artifacts-dev')  # имя бакета

# Смонтированный путь к бакету (если GCSFuse подключен): /content/artifacts -> gs://pik-artifacts-dev/
MOUNTED_ARTIFACTS = '/content/artifacts'

def _which(x: str) -> bool:
    from shutil import which
    return which(x) is not None

def _count_local(root: str) -> int:
    return len(glob.glob(os.path.join(root, '**', '*'), recursive=True))

# Проверки
if not Path(LOCAL_REGIONS).exists():
    raise SystemExit(f"[fatal] LOCAL_REGIONS не найден: {LOCAL_REGIONS}. Сначала запусти детекцию.")

# Соберём чуть статистики
local_files = _count_local(LOCAL_REGIONS)
units = sorted({Path(p).parts[-3] for p in glob.glob(os.path.join(LOCAL_REGIONS, '**', 'regions', 'region-*.json'), recursive=True)})
print('[local] units:', units[:8], ('...' if len(units) > 8 else ''))
print('[local] files:', local_files)

# Назначение
if RUN_TAG:
    dst_suffix = f"{DEST_PREFIX}/{RUN_TAG}"
else:
    dst_suffix = DEST_PREFIX

# Вариант 1: GCSFuse (копирование в смонтированный путь)
mounted_root = Path(MOUNTED_ARTIFACTS)
if mounted_root.exists():
    dest_dir = mounted_root / dst_suffix
    dest_dir.mkdir(parents=True, exist_ok=True)

    # rsync внутри локальной FS — быстро и просто
    # -a (архив), -v (verbose); заменит/удалит изменённые/удалённые при -r/-d
    # оставим без -d (delete), чтобы не удалять предыдущие загрузки в подпапке
    cmd = f"rsync -av '{LOCAL_REGIONS}/' '{dest_dir}/'"
    print('> using fuse:', cmd)
    t0 = time.time()
    subprocess.run(['bash','-lc', cmd], check=True)
    dt = time.time() - t0

    # Быстрый подсчёт
    remote_files = len(glob.glob(os.path.join(str(dest_dir), '**', '*'), recursive=True))
    print(f"[done] fuse sync -> {dest_dir} | files={remote_files} | {dt:.1f}s")

# Вариант 2: gsutil rsync (при отсутствии GCSFuse)
else:
    if not _which('gsutil'):
        raise SystemExit("[fatal] Ни GCSFuse (/content/artifacts), ни gsutil не найдены. Нечем выгружать.")
    dest_uri = f"gs://{GCS_BUCKET}/{dst_suffix}"
    # -m: мультипоточно; -r: рекурсивно; без -d, чтобы не удалять прошлое
    cmd = f"gsutil -m rsync -r '{LOCAL_REGIONS}' '{dest_uri}'"
    print('> using gsutil:', cmd)
    t0 = time.time()
    subprocess.run(['bash','-lc', cmd], check=True)
    dt = time.time() - t0
    print(f"[done] gsutil rsync -> {dest_uri} | {dt:.1f}s")

print('[hint] Регионы доступны под префиксом:', dst_suffix)


[local] units: ['PIK - Expert Guide - Platform IT Architecture - Assessment - v01', 'PIK - Platform IT Architecture Canvas - Table View - v01', 'PIK - Platform IT Architecture Canvases - v01', 'PIK 5-0 - Ecosystem Forces Scan - ENG', 'PIK 5-0 - Longtail Discovery - ENG', 'PIK 5-0 - MVP - ENG', 'PIK 5-0 - Market Sizing - ENG', 'PIK 5-0 - NFX Reinforcement Engines - ENG'] ...
[local] files: 348
> using fuse: rsync -av '/content/grounded_regions/' '/content/artifacts/grounded_regions/'
[done] fuse sync -> /content/artifacts/grounded_regions | files=368 | 8.5s
[hint] Регионы доступны под префиксом: grounded_regions


In [31]:
#@title Upload Cell Logs to GCS (robust: autodetect LOG_DIR, fuse/gsutil)
require_start()
import os, json, time, glob, subprocess
from pathlib import Path

# 0) Найти локальный LOG_DIR с cells.jsonl (а НЕ в /content/artifacts)
def _pick_log_dir():
    # 0.1 Явно заданная переменная
    if 'LOG_DIR' in globals():
        p = Path(globals()['LOG_DIR'])
        if p.exists(): return p
    # 0.2 RUN_ID из окружения или из ранее заданного
    rid = os.environ.get('RUN_ID', None)
    if rid:
        p = Path('/content/colab_runs') / rid
        if p.exists(): return p
    # 0.3 Самый свежий подкаталог в /content/colab_runs
    root = Path('/content/colab_runs')
    if root.exists():
        dirs = sorted([d for d in root.iterdir() if d.is_dir()],
                      key=lambda d: d.stat().st_mtime, reverse=True)
        if dirs: return dirs[0]
    # 0.4 В крайнем случае — создаём новый
    rid = time.strftime('%Y%m%d-%H%M%S')
    p = Path('/content/colab_runs') / rid
    p.mkdir(parents=True, exist_ok=True)
    return p

LOG_DIR = _pick_log_dir()
LOG_DIR = Path(LOG_DIR)
cells_path = LOG_DIR / 'cells.jsonl'

# 1) Если лог не появился — создадим пустой, чтобы не падать
if not cells_path.exists():
    print(f"[warn] cells.jsonl not found at {cells_path} — создаю пустой (запусти любую ячейку после логгера, чтобы появились записи).")
    cells_path.write_text('', encoding='utf-8')

# 2) Служебный run_info.json
info = {
    'run_id': LOG_DIR.name,
    'notebook_version': globals().get('NOTEBOOK_VERSION', 'unknown'),
    'notebook_updated': globals().get('NOTEBOOK_UPDATED', 'unknown'),
    'timestamp_utc': time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime()),
    'log_dir': str(LOG_DIR),
    'files_in_log_dir': len(list(LOG_DIR.glob('**/*'))),
}
info_path = LOG_DIR / 'run_info.json'
info_path.write_text(json.dumps(info, ensure_ascii=False, indent=2), encoding='utf-8')
print('[log] run_info.json:', info_path)

def _which(name: str) -> bool:
    from shutil import which
    return which(name) is not None

# 3) Назначение в бакете
GCS_BUCKET  = os.environ.get('GCS_BUCKET', 'pik-artifacts-dev')
DEST_PREFIX = os.environ.get('DEST_PREFIX', 'colab_runs')
RUN_ID_STR  = LOG_DIR.name
DEST_SUBDIR = f'{DEST_PREFIX}/{RUN_ID_STR}'

# 4) Путь через FUSE (если смонтирован /content/artifacts)
ARTIFACTS_MOUNT = Path('/content/artifacts')
if ARTIFACTS_MOUNT.exists():
    dst_dir = ARTIFACTS_MOUNT / DEST_SUBDIR
    dst_dir.mkdir(parents=True, exist_ok=True)
    # rsync всего подкаталога (cells.jsonl + run_info.json + проч.)
    cmd = f"rsync -av '{LOG_DIR}/' '{dst_dir}/'"
    print('> using fuse:', cmd)
    t0 = time.time()
    subprocess.run(['bash','-lc', cmd], check=True)
    dt = time.time() - t0
    remote_count = len(list(dst_dir.glob('**/*')))
    print(f"[done] fuse sync -> {dst_dir} | files={remote_count} | {dt:.1f}s")
    print('Remote path (fuse):', dst_dir)
else:
    # 5) Фолбэк — gsutil
    if not _which('gsutil'):
        raise SystemExit('[fatal] Ни FUSE (/content/artifacts), ни gsutil не найдены — лог некуда выгрузить.')
    dest_uri = f"gs://{GCS_BUCKET}/{DEST_SUBDIR}"
    print('> using gsutil: upload log files')
    t0 = time.time()
    subprocess.run(['bash','-lc', f"gsutil -h 'Content-Type:application/json' cp -r '{LOG_DIR}/cells.jsonl' '{dest_uri}/'"], check=True)
    subprocess.run(['bash','-lc', f"gsutil -h 'Content-Type:application/json' cp -r '{LOG_DIR}/run_info.json' '{dest_uri}/'"], check=True)
    dt = time.time() - t0
    print(f"[done] gsutil cp -> {dest_uri} | {dt:.1f}s")
    print('Remote path (gsutil):', dest_uri)

print('[hint] LOG_DIR =', LOG_DIR)


[warn] cells.jsonl not found at /content/artifacts/colab_runs/20250916-165657/cells.jsonl — создаю пустой (запусти любую ячейку после логгера, чтобы появились записи).
[log] run_info.json: /content/artifacts/colab_runs/20250916-165657/run_info.json
> using fuse: rsync -av '/content/artifacts/colab_runs/20250916-165657/' '/content/artifacts/colab_runs/20250916-165657/'
[done] fuse sync -> /content/artifacts/colab_runs/20250916-165657 | files=3 | 0.1s
Remote path (fuse): /content/artifacts/colab_runs/20250916-165657
[hint] LOG_DIR = /content/artifacts/colab_runs/20250916-165657


In [33]:
#@title Full Run — enumerate playbooks and frames; render all pages

require_start()
import os, re, glob, json, subprocess
from pathlib import Path

PLAYBOOKS_DIRS = ['/content/src_gcs/playbooks','/content/playbooks','/content/drive/MyDrive/playbooks']
FRAMES_DIRS   = ['/content/src_gcs/frames','/content/frames','/content/drive/MyDrive/frames']
OUT_PAGES_DIR = '/content/pages'
MANIFEST      = '/content/full_run_manifest.jsonl'
Path(OUT_PAGES_DIR).mkdir(parents=True, exist_ok=True)

def _first_existing(dirs):
  for d in dirs:
    if Path(d).exists():
      return d
  return None

pb_root = _first_existing(PLAYBOOKS_DIRS)
fr_root = _first_existing(FRAMES_DIRS)
print('playbooks root =', pb_root)
print('frames root    =', fr_root)

def _pdf_pages(pdf_path: str) -> int:
  try:
    out = subprocess.check_output(['pdfinfo', pdf_path], text=True)
    m = re.search(r'^Pages:\s+(\d+)', out, re.M)
    return int(m.group(1)) if m else 0
  except Exception:
    return 0

images = []
if pb_root:
  for pdf in sorted(glob.glob(os.path.join(pb_root, '*.pdf'))):
    name = Path(pdf).stem
    out_dir = Path(OUT_PAGES_DIR)/name
    out_dir.mkdir(parents=True, exist_ok=True)
    n = _pdf_pages(pdf)
    print('[render]', name, ': pages=', n)
    for p in range(1, n+1):
      png = out_dir/f'page-{p}.png'
      if not png.exists():
        subprocess.run(['pdftoppm','-f',str(p),'-l',str(p),'-png','-singlefile','-r','150', pdf, str(png.with_suffix(''))], check=True)
      images.append(str(png))

if fr_root:
  for ext in ('*.png','*.jpg','*.jpeg'):
    for fp in sorted(glob.glob(os.path.join(fr_root, '**', ext), recursive=True)):
      images.append(fp)

with open(MANIFEST,'w') as f:
  for im in images:
    f.write(json.dumps({'image': im})+'\n')

print('Prepared images:', len(images))
print('Manifest written to:', MANIFEST)

playbooks root = /content/src_gcs/playbooks
frames root    = /content/src_gcs/frames
[render] 2023-06 - fastbreakOne - Expert Guide - Ecosystem Strategy  - English : pages= 62
[render] PIK - Expert Guide - Platform IT Architecture - Playbook - v11 : pages= 50
[render] PIK 5-0 - Introduction - English : pages= 49
Prepared images: 186
Manifest written to: /content/full_run_manifest.jsonl


In [35]:
#@title Batch Detect — iterate manifest and write regions (playbooks+frames)
require_start()
import os, json, base64, shutil, traceback
from pathlib import Path
from PIL import Image

# Константы путей
MANIFEST   = '/content/full_run_manifest.jsonl'
DETECT_OUT = '/content/grounded_regions'

# Корни, по которым определяем unit (чтоб группировать выход)
PAGES_PB   = '/content/pages_src/playbooks'
PAGES_FR   = '/content/pages_src/frames'
FRAMES_RAW = '/content/src_gcs/frames'  # смонтированный источник PNG

Path(DETECT_OUT).mkdir(parents=True, exist_ok=True)

# Загрузка манифеста
images = []
mp = Path(MANIFEST)
if not mp.exists():
    raise SystemExit(f'[fatal] manifest not found: {MANIFEST}')
with mp.open('r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line: continue
        try:
            images.append(json.loads(line)['image'])
        except Exception:
            pass

def _unit_from_image(img: str) -> str:
    p = Path(img)
    s = str(p)
    if s.startswith(PAGES_PB):
        # /content/pages_src/playbooks/<doc>/page-*.png
        parts = p.parts
        idx = len(Path(PAGES_PB).parts)
        doc = parts[idx] if len(parts) > idx else p.stem
        return f'playbooks/{doc}'
    if s.startswith(PAGES_FR):
        # /content/pages_src/frames/<doc>/page-*.png
        parts = p.parts
        idx = len(Path(PAGES_FR).parts)
        doc = parts[idx] if len(parts) > idx else p.stem
        return f'frames-pdf/{doc}'
    if s.startswith(FRAMES_RAW):
        # /content/src_gcs/frames/**/<name>.png → использовать <parent>/<stem> если есть parent
        parent = p.parent.name
        if parent and parent not in ('frames',):
            return f'frames/{parent}-{p.stem}'
        return f'frames/{p.stem}'
    return 'misc'

def _ensure_dir(d: Path):
    d.mkdir(parents=True, exist_ok=True)

def _png_b64(img_path: str) -> str:
    try:
        im = Image.open(img_path).convert('RGB')
        import io
        buf = io.BytesIO()
        im.save(buf, format='PNG')
        return base64.b64encode(buf.getvalue()).decode('utf-8')
    except Exception:
        return ''

def _placeholder_detect(img_path: str, unit_dir: Path):
    rdir = unit_dir / 'regions'
    _ensure_dir(rdir)
    # region-1.json
    b64 = _png_b64(img_path)
    (rdir / 'region-1.json').write_text(
        json.dumps({'bbox': {'x':0,'y':0,'w':-1,'h':-1},
                    'text': '',
                    'image_b64': b64}, ensure_ascii=False),
        encoding='utf-8'
    )
    # preview region-1.png
    try:
        dst = rdir / 'region-1.png'
        if not dst.exists():
            shutil.copy2(img_path, dst)
    except Exception:
        pass

# Детектор если объявлен в ноутбуке
det_fn = globals().get('detect_image_to_regions') or globals().get('run_detection_one')

errors = 0
for i, im in enumerate(images, start=1):
    try:
        unit = _unit_from_image(im)
        unit_dir = Path(DETECT_OUT) / unit
        if callable(det_fn):
            try:
                det_fn(im, str(unit_dir))  # ожидаем сигнатуру (image_path, out_dir)
            except TypeError:
                # на всякий случай поддержим вариант (image_path) и пишем в unit_dir/regions внутри
                det_fn(im)
        else:
            _placeholder_detect(im, unit_dir)
        if (i % 20) == 0:
            print('.. processed:', i, '/', len(images))
    except Exception as e:
        errors += 1
        print('[error] on', im, ':', e)
        traceback.print_exc()

print('[done] images:', len(images), 'errors:', errors)
print('[local regions root]:', DETECT_OUT)


.. processed: 20 / 186
.. processed: 40 / 186
.. processed: 60 / 186
.. processed: 80 / 186
.. processed: 100 / 186
.. processed: 120 / 186
.. processed: 140 / 186
.. processed: 160 / 186
.. processed: 180 / 186
[done] images: 186 errors: 0
[local regions root]: /content/grounded_regions


In [36]:
#@title Run Summary — counts and outputs
require_start()
import os, glob, json
from pathlib import Path

MANIFEST   = '/content/full_run_manifest.jsonl'
DETECT_OUT = '/content/grounded_regions'
PAGES_ROOT = '/content/pages_src'  # включает /playbooks и /frames
ARTIFACTS_MOUNT = Path('/content/artifacts')
GCS_BUCKET = os.environ.get('GCS_BUCKET', 'pik-artifacts-dev')
DEST_PREFIX= os.environ.get('DEST_PREFIX', 'colab_runs')

# Манифест
images_total = 0
mp = Path(MANIFEST)
if mp.exists():
    with mp.open('r', encoding='utf-8') as f:
        for line in f:
            if line.strip(): images_total += 1

# Страницы
rendered_pages = len(glob.glob(os.path.join(PAGES_ROOT, '**', 'page-*.png'), recursive=True))

# Регионы
region_jsons  = len(glob.glob(os.path.join(DETECT_OUT, '**', 'regions', 'region-*.json'), recursive=True))
region_pngs   = len(glob.glob(os.path.join(DETECT_OUT, '**', 'regions', 'region-*.png'), recursive=True))

# Unit’ы (папки непосредственно под grounded_regions)
units = []
gr = Path(DETECT_OUT)
if gr.exists():
    for d in gr.iterdir():
        if d.is_dir():
            units.append(d.name)
units_sorted = sorted(units)

print('--- Run Summary ---')
print('Images in manifest        :', images_total)
print('Rendered pages PNG        :', rendered_pages)
print('Region JSON files         :', region_jsons)
print('Region preview PNG files  :', region_pngs)
print('Units (sample)            :', units_sorted[:10], ('...' if len(units_sorted)>10 else ''))
print('Local regions root        :', DETECT_OUT)

if ARTIFACTS_MOUNT.exists():
    print('Artifacts (fuse) root     :', ARTIFACTS_MOUNT)
    # Подсказка куда чаще всего выгружаем регионы через Upload Regions
    run_id = os.environ.get('RUN_ID', '(set RUN_ID for subdir)')
    print('Hint remote prefix        :', ARTIFACTS_MOUNT / f'grounded_regions')
else:
    print('Artifacts (fuse) root     : not mounted')
    print('Hint remote URI           :', f'gs://{GCS_BUCKET}/grounded_regions')


--- Run Summary ---
Images in manifest        : 186
Rendered pages PNG        : 190
Region JSON files         : 113
Region preview PNG files  : 113
Units (sample)            : ['PIK - Expert Guide - Platform IT Architecture - Assessment - v01', 'PIK - Platform IT Architecture Canvas - Table View - v01', 'PIK - Platform IT Architecture Canvases - v01', 'PIK 5-0 - Ecosystem Forces Scan - ENG', 'PIK 5-0 - Longtail Discovery - ENG', 'PIK 5-0 - MVP - ENG', 'PIK 5-0 - Market Sizing - ENG', 'PIK 5-0 - NFX Reinforcement Engines - ENG', 'PIK 5-0 - Network Effects Stimulation - ENG', 'PIK 5-0 - Platform AARRR Funnel - ENG'] ...
Local regions root        : /content/grounded_regions
Artifacts (fuse) root     : /content/artifacts
Hint remote prefix        : /content/artifacts/grounded_regions


In [ ]:
#@title Batch Detect — iterate manifest and write regions (+optional GCS upload)

require_start()
import os, json, shutil, base64
from pathlib import Path
from PIL import Image

MANIFEST = '/content/full_run_manifest.jsonl'
DETECT_OUT = '/content/grounded_regions'
PROMPTS = ['diagram','canvas','table','legend','arrow','node']
Path(DETECT_OUT).mkdir(parents=True, exist_ok=True)

def _png_b64(img_path: str) -> str:
  try:
    im = Image.open(img_path).convert('RGB')
    import io
    buf = io.BytesIO(); im.save(buf, format='PNG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')
  except Exception:
    return ''

def _fallback_detect(img_path: str, unit_dir: Path):
  (unit_dir/'regions').mkdir(parents=True, exist_ok=True)
  reg = unit_dir/'regions'/'region-1.json'
  b64 = _png_b64(img_path)
  reg.write_text(json.dumps({
    'bbox': {'x':0,'y':0,'w':-1,'h':-1},
    'text': '',
    'image_b64': b64,
  }), encoding='utf-8')
  # copy preview
  try:
    dst = unit_dir/'regions'/'region-1.png'
    if not dst.exists():
      shutil.copy2(img_path, dst)
  except Exception:
    pass

def _unit_from_path(p: Path) -> str:
  # Use parent dir for pages/<doc>/page-<n>.png; else fall back to stem
  if p.parent.name.startswith('page-'):
    return p.parent.parent.name
  if p.parent.name and p.parent.parent.name == 'pages':
    return p.parent.name
  return p.stem

def detect_image(img_path: str):
  # Try to call a notebook-level detect function if present; else fallback placeholder
  nb = globals()
  unit = _unit_from_path(Path(img_path))
  unit_dir = Path(DETECT_OUT)/unit
  fn = nb.get('detect_image_to_regions') or nb.get('run_detection_one')
  if callable(fn):
    try:
      fn(img_path, str(unit_dir))
      return
    except Exception as e:
      print('[warn] detect func failed, fallback:', e)
  _fallback_detect(img_path, unit_dir)

images = []
with open(MANIFEST, 'r', encoding='utf-8') as f:
  for line in f:
    if line.strip():
      obj = json.loads(line)
      images.append(obj.get('image'))

print('Detecting over', len(images), 'images')
for i, im in enumerate(images, start=1):
  try:
    detect_image(im)
    if i % 20 == 0:
      print('..', i, 'done')
  except Exception as e:
    print('[error] failed on', im, e)

print('Local regions root:', DETECT_OUT)

# Optional: upload to GCS if bucket var present
try:
  import subprocess
  bucket = GCS_BUCKET if 'GCS_BUCKET' in globals() else None
  if bucket:
    print('[upload] syncing to', f'gs://{bucket}/grounded_regions')
    subprocess.run(['bash','-lc', f"gsutil -m rsync -r '{DETECT_OUT}' 'gs://{bucket}/grounded_regions'"], check=False)
except Exception as e:
  print('[warn] GCS upload failed:', e)
